In [1]:
rm(list = ls()) # Borro todos los objetos
gc() # Garbage Collection

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,632258,33.8,1413441,75.5,1137648,60.8
Vcells,1170701,9.0,8388608,64.0,1815614,13.9


In [2]:
require("data.table")
require("rpart")
require("parallel")

Loading required package: data.table

Loading required package: rpart

Loading required package: parallel



In [3]:
PARAM <- list()
# reemplazar por las propias semillas
PARAM$semillas <- c(110251, 190283, 340211, 577463, 880249)

#------------------------------------------------------------------------------
# particionar agrega una columna llamada fold a un dataset
#  que consiste en una particion estratificada segun agrupa
# particionar( data=dataset, division=c(70,30), agrupa=clase_ternaria, seed=semilla)
#   crea una particion 70, 30

In [4]:
particionar <- function(data, division=c(70,30), agrupa = clase_ternaria, campo = "fold", start = 1, seed = semilla) {
  if (!is.na(seed)) set.seed(seed)

  bloque <- unlist(mapply(function(x, y) {
    rep(y, x)
  }, division, seq(from = start, length.out = length(division))))

  data[, (campo) := sample(rep(bloque, ceiling(.N / length(bloque))))[1:.N],
    by = agrupa
  ]
}

In [5]:
ArbolEstimarGanancia <- function(semilla, param_basicos) {
  # particiono estratificadamente el dataset
  particionar(dataset, division = c(7, 3), agrupa = "clase_ternaria", seed = semilla)

  # genero el modelo
  # quiero predecir clase_ternaria a partir del resto
  modelo <- rpart("clase_ternaria ~ .",
    data = dataset[fold == 1], # fold==1  es training,  el 70% de los datos
    xval = 0,
    control = param_basicos
  ) # aqui van los parametros del arbol

  # aplico el modelo a los datos de testing
  prediccion <- predict(modelo, # el modelo que genere recien
    dataset[fold == 2], # fold==2  es testing, el 30% de los datos
    type = "prob"
  ) # type= "prob"  es que devuelva la probabilidad

  # prediccion es una matriz con TRES columnas,
  #  llamadas "BAJA+1", "BAJA+2"  y "CONTINUA"
  # cada columna es el vector de probabilidades


  # calculo la ganancia en testing  qu es fold==2
  ganancia_test <- dataset[
    fold == 2,
    sum(ifelse(prediccion[, "BAJA+2"] > 0.025,
      ifelse(clase_ternaria == "BAJA+2", 117000, -3000),
      0
    ))
  ]

  # escalo la ganancia como si fuera todo el dataset
  ganancia_test_normalizada <- ganancia_test / 0.3

  return(ganancia_test_normalizada)
}

In [6]:
ArbolesMontecarlo <- function(semillas, param_basicos) {
  # la funcion mcmapply  llama a la funcion ArbolEstimarGanancia
  #  tantas veces como valores tenga el vector  PARAM$semillas
  ganancias <- mcmapply(ArbolEstimarGanancia,
    semillas, # paso el vector de semillas
    MoreArgs = list(param_basicos), # aqui paso el segundo parametro
    SIMPLIFY = FALSE,
    mc.cores = 5 # en Windows este valor debe ser 1
  )

  ganancia_promedio <- mean(unlist(ganancias))

  return(ganancia_promedio)
}

In [7]:
# Aqui se debe poner la carpeta de la computadora local
setwd("~/buckets/b1/") # Establezco el Working Directory
# cargo los datos

# cargo los datos
dataset <- fread("./datasets/dataset_pequeno.csv")

# trabajo solo con los datos con clase, es decir 202107
dataset <- dataset[clase_ternaria != ""]

# genero el archivo para Kaggle
# creo la carpeta donde va el experimento
# HT  representa  Hiperparameter Tuning
dir.create("./exp/", showWarnings = FALSE)
dir.create("./exp/HT2020/", showWarnings = FALSE)
archivo_salida <- "./exp/HT2020/gridsearch.txt"

In [8]:
# Generate the data.table to store the results of the Grid Search
tb_grid_search <- data.table(
  max_depth = integer(),
  min_split = integer(),
  cp = numeric(),
  min_bucket = numeric(),
  ganancia_promedio = numeric()
)

# Iterate through the nested loops for each hyperparameter
for (vmax_depth in c(17)) {
  for (vmin_split in c(2002)) {
    for (vcp in c(-0.99)) {
      for (vmin_bucket in  (998) )  {
        # Notice how parameters are set

        # minsplit: minimum number of records in a node to split
        param_basicos <- list(
          "cp" = vcp,                 # minimum complexity
          "minsplit" = vmin_split,
          "minbucket" = vmin_bucket,  # minimum number of records in a leaf
          "maxdepth" = vmax_depth    # maximum depth of the tree
        )

        # A single call, with seed 17
        ganancia_promedio <- ArbolesMontecarlo(PARAM$semillas, param_basicos)

        # Add to the table
        tb_grid_search <- rbindlist(
          list(tb_grid_search,
               list(vmax_depth, vmin_split, vcp, vmin_bucket, ganancia_promedio))
        )
      }
    }
    # Write the table to disk in each iteration of the outer loop
    Sys.sleep(2)  # Wait a couple of seconds
    fwrite(tb_grid_search,
           file = archivo_salida,
           sep = "\t")
  }
}
